# Dense dsm step by step from sensor images

This notebook correspond to the sensors to dense dsm pipeline, starting from epipolar a priori.

User has to have previously generated epipolar a priori with a run of SensorsToDenseDSM pipeline or SensorsToSparseDSM pipeline.


### Imports

In [ ]:
# Notebook local imports
import os
import math
###
import warnings
# Filter warnings
warnings.filterwarnings("ignore",category=UserWarning)
# import pretty print
import pprint 
pp = pprint.PrettyPrinter(indent=4)

# import external function for notebook
from notebook_helpers import get_full_data, show_data, save_data, get_dir_path, set_up_demo_inputs
from notebook_helpers import update_advanced_conf_with_a_priori, extract_a_priori_from_config, apply_grid_correction
from notebook_helpers import compute_cell, show_epipolar_images


In [ ]:
# Additional imports
from collections import OrderedDict
import numpy as np
import json
import matplotlib.pyplot as plt

In [ ]:

# CARS imports

# Applications
from cars.applications.application import Application
from cars.applications.sparse_matching import sparse_matching_tools

# Pipelines
import cars.pipelines.parameters.sensor_inputs_constants as sens_cst
from cars.pipelines.parameters import sensor_inputs, output_parameters
from cars.pipelines.pipeline_constants import GEOMETRY_PLUGIN

# Conf, core, orchestrator
from cars.core import cars_logging
from cars.core import inputs, preprocessing, roi_tools, tiling
from cars.core.utils import safe_makedirs
from cars.orchestrator import orchestrator
from cars.core.utils import make_relative_path_absolute
from cars import import_plugins

In [ ]:
# Show CARS version
from cars import __version__
print("CARS version used : {}".format(__version__))

In [ ]:
# Import external plugins
import_plugins()

---------

## Inputs/Outputs

### Define outputs

In [ ]:
# Modify with your own output path if needed
output_dir = os.path.join(get_dir_path(), "output_notebook")
safe_makedirs(output_dir)
print(output_dir)

## CARS Configuration 

More information for CARS configuration [here](https://cars.readthedocs.io/en/stable/user_guide/configuration.html)

### Define inputs

In [ ]:
# By default, the tutorial use data_gizeh_small.tar.bz2
input_dir_path = set_up_demo_inputs("data_gizeh_small")

inputs_conf = {
    "sensors": {
        "left": {
            "image": os.path.join(input_dir_path, "img1.tif"),
            "geomodel": {
              "path": os.path.join(input_dir_path, "img1.geom")
            },
            "color": os.path.join(input_dir_path, "color1.tif"),
            "classification": os.path.join(input_dir_path, "classif1.tif")
        },
        "right": {
            "image": os.path.join(input_dir_path, "img2.tif"),
            "geomodel": {
              "path": os.path.join(input_dir_path, "img2.geom")
            },
            "classification": os.path.join(input_dir_path, "classif2.tif")
        },   
    },
    "pairing": [["left", "right"]],
    "roi":{
      "type": "FeatureCollection",
      "features": [
        {
          "type": "Feature",
          "properties": {},
          "geometry": {
            "coordinates": [
              [
                [
                  31.134842671152256,
                  29.978783846127072
                ],
                [
                  31.134842671152256,
                  29.977029249051853
                ],
                [
                  31.137048614704412,
                  29.977029249051853
                ],
                [
                  31.137048614704412,
                  29.978783846127072
                ],
                [
                  31.134842671152256,
                  29.978783846127072
                ]
              ]
            ],
            "type": "Polygon"
          }
        }
      ]
    }
}

# Be carefull, pair names must be the same, if not, do not use this function, overide directly in your configuration
advanced_conf = {}
update_advanced_conf_with_a_priori(advanced_conf, os.path.join(input_dir_path, "used_conf.json"), input_dir_path)

#### Extract a priori from conf

In [ ]:
grid_coefficients, disparity_range, dem_median, dem_min, dem_max = extract_a_priori_from_config(advanced_conf)

In [ ]:
updated_inputs_conf = sensor_inputs.sensors_check_inputs(inputs_conf)
pp.pprint(updated_inputs_conf)

In [ ]:
updated_inputs_conf["initial_elevation"]["dem"] = dem_median
(
    _,
    updated_inputs_conf[GEOMETRY_PLUGIN],
    geom_plugin_without_dem_and_geoid,
    geom_plugin_with_dem_and_geoid,
    _
) = sensor_inputs.check_geometry_plugin(
    updated_inputs_conf, {}, updated_inputs_conf.get(GEOMETRY_PLUGIN, None)
)

---------

## Applications Init

#### GridGeneration

This application generates epipolar grids corresponding to sensor pair

In [ ]:
epipolar_grid_generation_application = Application("grid_generation")

#### Resampling

This application generates epipolar images from epipolar grids 

In [ ]:
resampling_application = Application("resampling")

#### DenseMatching

This application generates dense matches of stereo images pairs

In [ ]:
conf_pandora = OrderedDict([
    ('input',{'nodata_left': -9999, 'nodata_right': -9999}),
    ('pipeline',{
        'matching_cost': {
            'matching_cost_method': 'census',
            'window_size': 5,
            'subpix': 1,
        },
        'optimization': {
            'optimization_method': 'sgm',
            'overcounting': False,
            'penalty': {
                'P1': 8,
                'P2': 32,
                'p2_method': 'constant',
                'penalty_method': 'sgm_penalty'
            },
            'sgm_version': 'c++',
            'min_cost_paths': False
        },
        'cost_volume_confidence': {
            'confidence_method': 'ambiguity',
            'eta_max': 0.7,
            'eta_step': 0.01,
           'indicator': ''
        },
        'disparity': {
            'disparity_method': 'wta',
            'invalid_disparity': np.nan
        },
        'refinement': {'refinement_method': 'vfit'},
        'filter': {'filter_method': 'median', 'filter_size': 3},
        'validation': {
            'validation_method': 'cross_checking_accurate',
            'cross_checking_threshold': 1.0
        }
    })
])

In [ ]:
conf_dense_matching = {'method': 'census_sgm',
                       'loader': 'pandora',
                       'loader_conf': conf_pandora,
                      }

In [ ]:
dense_matching_application = Application("dense_matching",cfg=conf_dense_matching)

#### Show used application configuration

In [ ]:
# Example with dense matching application
dense_matching_application.print_config()

#### Triangulation

This application triangulates matches, in order to get each (X, Y, Z) point position

In [ ]:
triangulation_application = Application("triangulation")

#### PointCloudFusion

This application performs the fusion of epipolar points from pairs to a terrain point cloud

In [ ]:
pc_fusion_application = Application("point_cloud_fusion")

#### PointCloudOutlierRemoval : small components

This application removes outliers points. The method used is "small component removal"

In [ ]:
conf_outlier_removal_small_components = {"method": "small_components", "activated": True}
pc_outlier_removal_small_comp_application = Application("point_cloud_outlier_removal", cfg=conf_outlier_removal_small_components)

#### PointCloudOutlierRemoval : statistical

This application removes outliers points. The method used is "statistical removal"

In [ ]:
conf_outlier_removal_small_statistical = {"method": "statistical", "activated": True}
pc_outlier_removal_stats_application = Application("point_cloud_outlier_removal", cfg=conf_outlier_removal_small_statistical)


#### PointCloudRasterization

This application performs the rasterization of a terrain point cloint.

In [ ]:
conf_rasterization = { 
    "method": "simple_gaussian",
    "dsm_radius": 3,
    "sigma": 0.3
}
rasterization_application = Application("point_cloud_rasterization", cfg=conf_rasterization)
rasterization_application.print_config()

### Create orchestrator


In [ ]:
# Customize orchestrator
orchestrator_conf = {"mode": "sequential"} 
cars_orchestrator = orchestrator.Orchestrator(orchestrator_conf=orchestrator_conf, out_dir=output_dir)

---------

## Run pipeline step by step from sensors to DSM

### Sensors images generation

From input configuration "inputs" seen before

In [ ]:
_, sensor_image_left, sensor_image_right = sensor_inputs.generate_inputs(updated_inputs_conf, geom_plugin_without_dem_and_geoid)[0]

### Generate ROI Polygon

In [ ]:
input_roi_poly, input_roi_epsg = roi_tools.generate_roi_poly_from_inputs(
    updated_inputs_conf[sens_cst.ROI]
)
print("ROI Polygon: {}\n".format(input_roi_poly))
print("EPSG: {}".format(input_roi_epsg))

### Grid Generation : epipolar grid generation

In [ ]:
grid_left, grid_right = epipolar_grid_generation_application.run(
    sensor_image_left,
    sensor_image_right,
    geom_plugin_with_dem_and_geoid,
    orchestrator=cars_orchestrator
)

### Correct right grid

Correction coefficients for the right grid (computed during the prepare step)

Generate corrected right grid

In [ ]:
print("Grid correction to apply : {}".format(grid_coefficients))
corrected_grid_right = apply_grid_correction(grid_right, grid_coefficients, output_dir)
corrected_grid_left = grid_left

### Define disparity interval

In [ ]:
dmin, dmax = disparity_range
print("Global Disparity min = {} , max = {}".format(dmin, dmax))

### Define local diparity range grid

In [ ]:
# Generate geometry loader with dem min and max and geoid
geom_plugin_with_dem_min_and_geoid = (
    sensor_inputs.generate_geometry_plugin_with_dem(
        updated_inputs_conf.get(GEOMETRY_PLUGIN, None),
        updated_inputs_conf,
        dem=dem_min,
    )
)
geom_plugin_with_dem_max_and_geoid = (
    sensor_inputs.generate_geometry_plugin_with_dem(
        updated_inputs_conf.get(GEOMETRY_PLUGIN, None),
        updated_inputs_conf,
        dem=dem_max,
    )
)

# Generate grids
disp_range_grid = dense_matching_application.generate_disparity_grids(
    sensor_image_right,
    corrected_grid_right,
    geom_plugin_with_dem_and_geoid,
    dem_min=dem_min,
    dem_max=dem_max,
    dem_median=dem_median,
)


### Compute margins used in dense matching, with corresponding disparity min and max

In [ ]:
dense_matching_margins_fun = dense_matching_application.get_margins_fun(
    corrected_grid_left, disp_range_grid)

#### Compute epipolar roi to use

In [ ]:
epipolar_roi = preprocessing.compute_epipolar_roi(
    input_roi_poly, 
    input_roi_epsg, 
    geom_plugin_with_dem_and_geoid, 
    sensor_image_left, 
    sensor_image_right, 
    grid_left, 
    corrected_grid_right,
    output_dir,
    disp_min=np.min(disp_range_grid[0, 0]["disp_min_grid"].values),
    disp_max=np.max(disp_range_grid[0, 0]["disp_max_grid"].values)
    )
print("Epipolar ROI: {}".format(epipolar_roi))

### Resampling : epipolar images generation


In [ ]:
(
    optimum_tile_size,
    local_tile_optimal_size_fun,
) = dense_matching_application.get_optimal_tile_size(
    disp_range_grid,
    cars_orchestrator.cluster.checked_conf_cluster[
        "max_ram_per_worker"
    ],
)

In [ ]:
epipolar_image_left, epipolar_image_right = resampling_application.run(
    sensor_image_left,
    sensor_image_right,
    grid_left,
    corrected_grid_right,
    orchestrator=cars_orchestrator,
    margins_fun=dense_matching_margins_fun,
    tile_width=optimum_tile_size,
    tile_height=optimum_tile_size,
    add_color=True,
    epipolar_roi=epipolar_roi
)
# Compute cell
compute_cell(cars_orchestrator, [epipolar_image_left, epipolar_image_right])

In [ ]:
data_color = get_full_data(epipolar_image_left, "color")
show_data(data_color, figsize=(4, 4),  mode='image')

In [ ]:
data_classif = get_full_data(epipolar_image_left, "classif")
print(data_classif.shape)
show_data(data_classif, figsize=(4, 4))

### Show epipolar image

In [ ]:
data_image_left = get_full_data(epipolar_image_left, "im")
data_mask_left = get_full_data(epipolar_image_left, "msk")
data_image_right = get_full_data(epipolar_image_right, "im")
data_mask_right = get_full_data(epipolar_image_right, "msk")

In [ ]:
show_epipolar_images(data_image_left, data_mask_left, data_image_right, data_mask_right)

### Dense Matching: compute disparities with pandora

In [ ]:
epipolar_disparity_map = dense_matching_application.run(
    epipolar_image_left,
    epipolar_image_right,
    local_tile_optimal_size_fun,
    orchestrator=cars_orchestrator,
    disp_range_grid=disp_range_grid,
)
               
# Compute cell              
compute_cell(cars_orchestrator, [epipolar_disparity_map])


#### Show full disparity map

In [ ]:
data_disparity = get_full_data(epipolar_disparity_map, "disp")
show_data(data_disparity, figsize=(4, 4))

 Compute epsg

In [ ]:
epsg = preprocessing.compute_epsg(
    sensor_image_left, 
    sensor_image_right,
    grid_left,
    corrected_grid_right,
    geom_plugin_with_dem_and_geoid,
    disp_min=np.min(disp_range_grid[0, 0]["disp_min_grid"].values),
    disp_max=np.max(disp_range_grid[0, 0]["disp_max_grid"].values)
)

### Triangulation : triangulate matches

In [ ]:
epipolar_point_cloud = triangulation_application.run(
    sensor_image_left,
    sensor_image_right,
    epipolar_image_left,
    grid_left,
    corrected_grid_right,
    epipolar_disparity_map,
    epsg,
    geom_plugin_without_dem_and_geoid,
    orchestrator=cars_orchestrator,
    uncorrected_grid_right=grid_right,
    geoid_path=updated_inputs_conf[sens_cst.INITIAL_ELEVATION][sens_cst.GEOID],
)
# Compute cell              
compute_cell(cars_orchestrator, [epipolar_point_cloud])



#### Compute terrain bounding box

In [ ]:
current_terrain_roi_bbox, _ = preprocessing.compute_terrain_bbox(
    sensor_image_left,
    sensor_image_right,
    epipolar_image_left,
    grid_left,
    corrected_grid_right,
    epsg,
    geom_plugin_with_dem_and_geoid,
    resolution=0.5,
    disp_min=np.min(disp_range_grid[0, 0]["disp_min_grid"].values),
    disp_max=np.max(disp_range_grid[0, 0]["disp_max_grid"].values),
    orchestrator=cars_orchestrator
)
# Compute roi polygon, in input EPSG
roi_poly = preprocessing.compute_roi_poly(
    input_roi_poly, input_roi_epsg, epsg
)
terrain_bounds, optimal_terrain_tile_width = preprocessing.compute_terrain_bounds(
    [current_terrain_roi_bbox],
    roi_poly=roi_poly,
    resolution=0.5
)

#### Transform point cloud to terrain point cloud

In [ ]:
merged_point_clouds = pc_fusion_application.run(
    [epipolar_point_cloud],
    terrain_bounds,
    epsg,
    orchestrator=cars_orchestrator,
    margins=rasterization_application.get_margins(0.5),
    optimal_terrain_tile_width=optimal_terrain_tile_width
)
# Compute cell              
compute_cell(cars_orchestrator, [merged_point_clouds])

### Point Cloud Outlier Removing : remove points with small components removing method

In [ ]:
filtered_sc_merged_point_clouds = pc_outlier_removal_small_comp_application.run(
    merged_point_clouds,
    orchestrator=cars_orchestrator,
)    
# Compute cell              
compute_cell(cars_orchestrator, [filtered_sc_merged_point_clouds])

### Point Cloud Outlier Removing: remove points with statistical removing method

In [ ]:
filtered_stats_merged_point_clouds = pc_outlier_removal_stats_application.run(
    filtered_sc_merged_point_clouds,
    orchestrator=cars_orchestrator,
)
# Compute cell              
compute_cell(cars_orchestrator, [filtered_stats_merged_point_clouds])

### Rasterization : rasterize point cloud

In [ ]:
dsm = rasterization_application.run(
    filtered_stats_merged_point_clouds,
    epsg,
    resolution=0.5,
    orchestrator=cars_orchestrator,
    color_dtype="uint16"
)
# Compute cell              
compute_cell(cars_orchestrator, [dsm])

### Show DSM

In [ ]:
data_dsm = get_full_data(dsm, "hgt")
show_data(data_dsm, figsize=(4, 4), mode="dsm")

### Show ortho image

In [ ]:
data_ortho = get_full_data(dsm, "img")
show_data(data_ortho, figsize=(4, 4), mode='image')

### Save DSM

In [ ]:
save_data(dsm, os.path.join(output_dir, "dsm.tif"), "hgt")